In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [5, 5, 1, 32]                                                         #先用32通道5*5卷积核
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),                 #权重的初始化方法采用截断的正太分布方法，一开始的方差设置为0.1
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])    
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))                          #偏置量的初始化方法采用固定值初始化，参数为0.1
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1                                #padding参数设置为SAME
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')                    #padding参数设置为Valid


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1),        #第二层的卷积核数量为64个
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2                                #padding参数设置为SAME
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss                                          #初始化正则因子为7e-5
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 100, entropy los

# 调整卷积核大小

将5*5卷积核变为4*4卷积核，而通道数不改变

In [2]:
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [4, 4, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([4, 4, 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 1.172320, l2_loss: 25242.560547, total loss: 2.939299
0.7
step 200, entropy loss: 0.827551, l2_loss: 25239.560547, total loss: 2.594320
0.73
step 300, entropy loss: 0.488565, l2_loss: 25236.380859, total loss: 2.255112
0.87
step 400, entropy loss: 0.398262, l2_loss: 25233.156250, total loss: 2.164583
0.87
step 500, entropy loss: 0.332912, l2_loss: 25229.837891, total loss: 2.099001
0.86
step 600, entropy loss: 0.398372, l2_loss: 25226.480469, total loss: 2.164226
0.88
step 700, entropy loss: 0.263070, l2_loss: 25223.113281, total loss: 2.028688
0.88
step 800, entropy loss: 0.270334, l2_loss: 25219.714844, total loss: 2.035714
0.95
step 900, entropy loss: 0.369047, l2_loss: 25216.328125

将卷积核大小调小之后，准确率变低了，原因可能是由于每一次卷积计算后携带的信息减少

增大卷积核大小，变为6*6卷积核

In [4]:

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  #先用32通道5*5卷积核
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

step 100, entropy loss: 0.786500, l2_loss: 50745.128906, total loss: 4.338659
0.77
step 200, entropy loss: 0.430121, l2_loss: 50738.117188, total loss: 3.981790
0.83
step 300, entropy loss: 0.544590, l2_loss: 50731.117188, total loss: 4.095768
0.84
step 400, entropy loss: 0.254161, l2_loss: 50724.125000, total loss: 3.804850
0.96
step 500, entropy loss: 0.303605, l2_loss: 50717.078125, total loss: 3.853800
0.89
step 600, entropy loss: 0.399024, l2_loss: 50710.062500, total loss: 3.948728
0.94
step 700, entropy loss: 0.263707, l2_loss: 50703.000000, total loss: 3.812917
0.95
step 800, entropy loss: 0.232686, l2_loss: 50695.945312, total loss: 3.781402
0.92
step 900, entropy loss: 0.162647, l2_loss: 50688.921875, total loss: 3.710871
0.95
step 1000, entropy loss: 0.198880, l2_loss: 50681.906250, total loss: 3.746613
0.93
0.9387
step 1100, entropy loss: 0.173385, l2_loss: 50674.859375, total loss: 3.720625
0.9
step 1200, entropy loss: 0.373714, l2_loss: 50667.804688, total loss: 3.920461


结果比5*5卷积核好了一点。